In [1]:
import os

def read_code_files(repo_path):
    code_snippets = []
    for root, dirs, files in os.walk(repo_path):
        for file in files:
            if file.endswith(('.py', '.js', '.java', '.cpp')):  # 根据需要添加文件类型
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    code_snippets.append(f.read())
    return code_snippets

code_snippets = read_code_files('/Users/yuanwen/Desktop/Docker_Environment/intern2/2/agent1/lungegfr')

/Users/yuanwen/anaconda3/envs/env_early/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/yuanwen/anaconda3/envs/env_early/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/yuanwen/anaconda3/envs/env_early/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/librav1e.0.6.dylib
  Referenced from: <33F98F4C-0602-3AD3-A584-BF1B105A8B67> /Users/yuanwen/anaconda3/envs/env_early/lib/libavif.16.1.1.dylib
  Reason: tried: '/Users/yuanwen/anaconda3/envs/env_early/lib/librav1e.0.6.dylib' (no such file), '/Users/yuanwen/anaconda3/envs/env_early/lib/python3.9/site-packages/torchvision/../../../librav1e.0.6.dylib' (no such file), '/Users/yuanwen/anaconda3/envs/env_early/lib/python3.9/site-packages/torchvisio

ImportError: cannot import name 'TFPreTrainedModel' from 'transformers' (/Users/yuanwen/anaconda3/envs/env_early/lib/python3.9/site-packages/transformers/__init__.py)

In [1]:
from src.services.llm import LLM

# from src.services.agents import Agent, Tool
from src.services.custom_tools import custom_tools
import glob
import os

os.environ["NO_PROXY"] = "*"

# llm = LLM(model_name="qwen3:8b")

llm = LLM(
    model_name="deepseek-chat",
    llm_url="https://api.deepseek.com/chat/completions",
    api_key="sk-b3cf6a0f243d43dc947417ecf08fedf6",
    format="deepseek-chat",
)
ct = custom_tools(llm=llm)

tools = [ct.func_cat, ct.func_ls, ct.func_python]


message = llm.query_with_tools(
    prompt="agent1/lungegfr/ is a GitHub package I downloaded. Please run tests and give me the prediction results. "
    "Any test code you write using tools should be placed under agent1/test/. ", max_steps=5, tools=tools, verbose=True
)

agent1/lungegfr/ is a GitHub package I downloaded. Please run tests and give me the prediction results. Any test code you write using tools should be placed under agent1/test/. 
[{'role': 'user', 'content': 'agent1/lungegfr/ is a GitHub package I downloaded. Please run tests and give me the prediction results. Any test code you write using tools should be placed under agent1/test/. '}]
I'll help you explore the lungegfr package and run tests. Let me first examine the structure of the package to understand what we're working with..............
Calling function 'func_ls' with arguments: {'filepath': 'agent1/lungegfr'}
[{'role': 'user', 'content': 'agent1/lungegfr/ is a GitHub package I downloaded. Please run tests and give me the prediction results. Any test code you write using tools should be placed under agent1/test/. '}, {'role': 'assistant', 'content': '', 'tool_calls': [{'id': 'call_00_FECycqfxMfBGndCHuzZTXMX9', 'type': 'function', 'function': {'name': 'func_ls', 'arguments': ''}, 

TypeError: src.services.custom_tools.custom_tools.func_ls() argument after ** must be a mapping, not str

In [1]:
from src.services.llm import LLM

# from src.services.agents import Agent, Tool
from src.services.custom_tools import custom_tools
import glob
import os

os.environ["NO_PROXY"] = "*"

# llm = LLM(model_name="qwen3:8b")

llm = LLM(
    model_name="deepseek-chat",
    llm_url="https://api.deepseek.com/chat/completions",
    api_key="sk-b3cf6a0f243d43dc947417ecf08fedf6",
    format="deepseek-chat",
)
def func1(a: float) -> float:
    return 2.0 * a

def func2(a: float) -> float:
    return 3.0 * a

tools = [func1, func2]

message = llm.query_duplicate(
    prompt="what is func1(10) + func2(20)", max_steps=2, tools=tools, verbose=True
)

what is func1(10) + func2(20)
I'll calculate func1(10) + func2(20) for you by calling both functions...............The result of func1(10) + func2(20) is 20.0 + 60.0 = **80.0**

In [2]:
import ollama
import requests
import os

os.environ["NO_PROXY"] = "*"

ollama.host = 'http://localhost:11434'

# 定义可用的工具
available_functions = {
    'request': requests.request,
}
tools = [
    {
        "name": "func1",
        "description": "Multiply input by 2",
        "parameters": [
            {"name": "a", "type": "int"}
        ]
    }
]

# 发送请求给模型

def func1(a: float) -> float:
    return 2.0 * a

def func2(a: float) -> float:
    return 3.0 * a


client = ollama.Client(host='http://localhost:11434')

tools = [
    {
        "type": "function",
        "function": {
            "name": "func1",
            "description": "Multiply input by 2",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "The input number"},
                },
                "required": ["a"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "func2",
            "description": "Multiply input by 3",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "The input number"},
                },
                "required": ["a"],
            },
        },
    },
]

response = client.chat(
    'qwen3:8b',
    messages=[{
        'role': 'user',
        'content': 'what is func1(10) + func2(20)',
    }],
    tools=tools,  # 将 requests.request 作为工具传递给模型
)


In [3]:
messages = [
    {"role": "user", "content": "what is func1(10) + func2(20)"}
]

tool_results = []
for call in response.message.tool_calls:
    func_name = call.function.name
    args = call.function.arguments

    # 执行函数
    if func_name == "func1":
        result = func1(**args)
    elif func_name == "func2":
        result = func2(**args)
    else:
        result = None

    # 使用正确的消息格式
    print(call)
    tool_results.append({
        "role": "tool",
        "content": str(result),
        "name": func_name,
        "tool_call_id": call.id  # 需要关联工具调用ID
    })

# 将工具执行结果添加到消息中
messages.extend(tool_results)

# 再次调用 LLM，让模型基于工具结果生成最终回答
response2 = client.chat('qwen3:8b', messages=messages)
print(response2.message.content)

function=Function(name='func1', arguments={'a': 10})


AttributeError: 'ToolCall' object has no attribute 'id'

In [17]:
response.message.tool_calls

[ToolCall(function=Function(name='func1', arguments={'a': 10})),
 ToolCall(function=Function(name='func2', arguments={'a': 20}))]

In [19]:
from openai import OpenAI
import os

# 设置 DeepSeek API Key
os.environ["OPENAI_API_KEY"] = "sk-b3cf6a0f243d43dc947417ecf08fedf6"
import json

# 初始化 DeepSeek 客户端
client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://api.deepseek.com"  # 指向 DeepSeek API
)

# 定义工具函数
def func1(a: float) -> float:
    return 2.0 * a

def func2(a: float) -> float:
    return 3.0 * a

tools = [
    {
        "type": "function",
        "function": {
            "name": "func1",
            "description": "Multiply input by 2",
            "parameters": {
                "type": "object",
                "properties": {"a": {"type": "number"}},
                "required": ["a"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "func2",
            "description": "Multiply input by 3",
            "parameters": {
                "type": "object",
                "properties": {"a": {"type": "number"}},
                "required": ["a"],
            },
        },
    },
]

# 第一次调用 - 获取工具调用
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "user", "content": "what is func1(10) + func2(20)?"}
    ],
    tools=tools,
    tool_choice="auto",
)

# 获取assistant的消息
assistant_message = response.choices[0].message
print("第一次响应:", assistant_message)

# 检查是否有工具调用
if assistant_message.tool_calls:
    messages = [
        {"role": "user", "content": "what is func1(10) + func2(20)?"},
        {"role": "assistant", "content": "", "tool_calls": assistant_message.tool_calls}
    ]
    
    tool_results = []
    
    for tool_call in assistant_message.tool_calls:
        func_name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
        
        # 执行对应的函数
        if func_name == "func1":
            result = func1(**args)
        elif func_name == "func2":
            result = func2(**args)
        else:
            result = None
        
        # 添加工具执行结果
        tool_results.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": func_name,
            "content": str(result)
        })
    
    # 添加工具结果到消息中
    messages.extend(tool_results)
    
    # 第二次调用 - 获取最终答案
    final_response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools,
    )
    
    print("最终答案:", final_response.choices[0].message.content)
else:
    print("没有工具调用:", assistant_message.content)

第一次响应: ChatCompletionMessage(content="I'll calculate func1(10) + func2(20) for you by calling both functions and then adding the results.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_00_mTrQzB0LFDPYcGPiFuptq1z0', function=Function(arguments='{"a": 10}', name='func1'), type='function', index=0), ChatCompletionMessageFunctionToolCall(id='call_01_Wix5Frk5FMfgLoguDRNpeKKq', function=Function(arguments='{"a": 20}', name='func2'), type='function', index=1)])
最终答案: The result of func1(10) + func2(20) is 20.0 + 60.0 = **80.0**

Here's the calculation:
- func1(10) = 10 × 2 = 20.0
- func2(20) = 20 × 3 = 60.0
- Sum = 20.0 + 60.0 = 80.0


In [20]:
messages

[{'role': 'user', 'content': 'what is func1(10) + func2(20)?'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [ChatCompletionMessageFunctionToolCall(id='call_00_mTrQzB0LFDPYcGPiFuptq1z0', function=Function(arguments='{"a": 10}', name='func1'), type='function', index=0),
   ChatCompletionMessageFunctionToolCall(id='call_01_Wix5Frk5FMfgLoguDRNpeKKq', function=Function(arguments='{"a": 20}', name='func2'), type='function', index=1)]},
 {'tool_call_id': 'call_00_mTrQzB0LFDPYcGPiFuptq1z0',
  'role': 'tool',
  'name': 'func1',
  'content': '20.0'},
 {'tool_call_id': 'call_01_Wix5Frk5FMfgLoguDRNpeKKq',
  'role': 'tool',
  'name': 'func2',
  'content': '60.0'}]

In [1]:
import os
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.tools import tool
from pathlib import Path
from langchain.schema import Document

os.environ.clear()
os.environ['NO_PROXY'] = 'localhost,127.0.0.1,::1'


# ========== 1. 定义函数 (Tools) ==========
@tool
def func1(a: int) -> int:
    """Multiply input by 2"""
    return 2 * a

@tool
def func2(a: int) -> int:
    """Multiply input by 3"""
    return 3 * a

tools = [func1, func2]
print("A")
# ========== 2. 构建数据库 (RAG) ==========

folder_path = "/Users/yuanwen/Desktop/Docker_Environment/intern2/2/agent1/lungegfr"

target_exts = [".py", ".md", ".txt", ".m"]
documents = []

# 遍历目录
for path in Path(folder_path).rglob("*"):
    if path.is_file():
        file_path = str(path.resolve())
        ext = path.suffix.lower()
        if ext in target_exts:
            # 读取内容
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
            except Exception:
                content = "read from file failed"
        else:
            content = "too large to read"  # 非目标文件内容为空
        
        content = f"# {file_path}\n\n{content}"

        # 构建 Document
        doc = Document(
            page_content=content,
            metadata={"source": file_path}
        )
        documents.append(doc)

print(f"总文件数: {len(documents)}")

# 向量化 + 存储
embeddings = OllamaEmbeddings(model="unclemusclez/jina-embeddings-v2-base-code:latest")
vectordb = Chroma.from_documents(documents, embeddings, persist_directory="./chroma_db")
retriever = vectordb.as_retriever(search_kwargs={"k":500})
print("B")
# ========== 3. LLM + 工具绑定 ==========
# callbacks = [StreamingStdOutCallbackHandler()]

# llm_with_tools = ChatOllama(
#     model="qwen3:8b",
#     streaming=True,
#     callback_manager=callbacks
# ).bind_tools(tools)
# print("C")
# # ========== 4. 构建带工具的 RAG QA ==========
# qa = RetrievalQA.from_chain_type(
#     llm=llm_with_tools,   # 这里用支持工具调用的 LLM
#     retriever=retriever,
#     chain_type="stuff"
# )
# print("D")
# # ========== 5. 使用示例 ==========
# # 问题中既涉及数据库又涉及函数
# query = "根据数据库内容, 请告诉我readme.txt的秘密"
# result = qa.invoke(query)

# print("综合结果:", result)

A
总文件数: 99
B


In [5]:
results = retriever.get_relevant_documents("""6.Run test_allpatient.py with python to generate the prediction results of all the input slices. (35 seconds are needed to run all the data)

7.Run Obtainresults.m with Matlab to obtain the predicted DLS of each patient. (25 seconds are needed to obtain the final results)
""")
print("检索到的文档数量:", len(results))
for doc in results[:1]:
    print(doc.metadata, doc.page_content[:200])

检索到的文档数量: 99
{'source': '/Users/yuanwen/Desktop/Docker_Environment/intern2/2/agent1/lungegfr/readme.txt'} # /Users/yuanwen/Desktop/Docker_Environment/intern2/2/agent1/lungegfr/readme.txt

1.Install python 3.5, keras 2.1.3, tensorflow 1.13.1, and Matlab 2019a (2h is needed to intall all softwares)

2.Downl


In [ ]:
import inspect
from typing import Callable, List, Dict, Any

def func1(a: int) -> int:
    """Multiply input by 2"""
    return 2 * a

def func2(a: int, b: str) -> int:
    """Multiply input by 3"""
    return 3 * a

create_tools([func1, func2])

[{'type': 'function',
  'function': {'name': 'func1',
   'description': 'Multiply input by 2',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number', 'description': 'Parameter a'}},
    'required': ['a']}}},
 {'type': 'function',
  'function': {'name': 'func2',
   'description': 'Multiply input by 3',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number', 'description': 'Parameter a'},
     'b': {'type': 'string', 'description': 'Parameter b'}},
    'required': ['a', 'b']}}}]